In [2]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

**Pra-processing Data** <br>
Dataset yang digunakan untuk proses training merupakan
hasil penggabungan file train yang telah disediakan. Pada
tahap ini, dilakukan ekstraksi fitur dengan memisahkan kolom
id menjadi pangan, provinsi, dan tanggal untuk mendapatkan
informasi yang lebih terstruktur. Selanjutnya, tanggal
dikonversi ke format datetime serta ditambahkan fitur year,
month, dan day. Hal ini dilakukan karena XGBoost bekerja
dengan data numerik dan mengandalkan perbedaan nilai antar
fitur untuk menemukan pola dalam data. Kemudian, untuk
menangani nilai yang hilang, kolom price diisi menggunakan
nilai median agar tidak terjadi distorsi pada model. Variabel
kategorik seperti pangan dan provinsi kemudian diencoding
menggunakan one-hot encoding agar dapat diproses sebagai
data numerik.

In [3]:
# Membaca dataset
df = pd.read_csv("final_dataset.csv")

# Pisahkan id
df[['pangan', 'provinsi', 'tanggal']] = df['id'].str.split('/', expand=True)
df['tanggal'] = pd.to_datetime(df['tanggal'], format='%Y-%m-%d')

# Tambahkan fitur tanggal: year, month, day
df['year'] = df['tanggal'].dt.year
df['month'] = df['tanggal'].dt.month
df['day'] = df['tanggal'].dt.day

# Isi misisng value menggunakan median
df.fillna({'price': df['price'].median()}, inplace=True)

# Encoding kategori menggunakan one-hot encoding
df = pd.get_dummies(df, columns=['pangan', 'provinsi'])

# Pisahkan fitur (X) dan target (y)
X = df.drop(columns=['id', 'tanggal', 'price'])
y = df['price']

**Pembagian Dataset** <br>
Setelah tahap pra-pemrosesan, dilakukan pemisahan data
dengan memisahkan fitur (x) dan target (y), di mana x berisi
semua fitur kecuali price, id, dan tanggal. Data kemudian
dibagi menjadi dua bagian yaitu data latih dan data uji dengan
rasio 80:20 menggunakan train-test split untuk memastikan
model dapat dievaluasi dengan baik. Selanjutnya, data
dikonversi ke dalam format DMatrix, yang merupakan
format yang lebih efisien untuk digunakan dalam XGBoost.

In [4]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=123)

# Konversi ke DMatrix untuk XGBoost
train_dmatrix = xgb.DMatrix(data=X_train, label=y_train)
test_dmatrix = xgb.DMatrix(data=X_test)

**Pemodelan dengan XGBoost** <br>
Pemodelan dilakukan menggunakan algoritma XGBoost
dengan parameter utama seperti objective yang diset ke
reg:squarederror untuk regresi, max_depth sebesar 7 agar
dapat menangkap kompleksitas data, learning_rate sebesar
0.05 agar model dapat belajar secara bertahap dan
menghindari overfitting, serta eval_metric menggunakan
rmse sebagai metrik evaluasi utama. Model dilatih selama
200 iterasi (num_round=200) dengan menggunakan seed = 42.

In [5]:
# Parameter XGBoost dengan seed
params = {
    'objective': 'reg:squarederror',
    'max_depth': 7,              
    'learning_rate': 0.05,       
    'eval_metric': 'rmse',
    'seed': 42, 
}

# Train model
num_round = 200
model = xgb.train(params, train_dmatrix, num_round)

**Evaluasi Model** <br>
Untuk mengevaluasi model, digunakan Root Mean
Squared Error (RMSE) dan Mean Absolute Error (MAE)
guna mengukur kinerja model. Selain itu, Selain itu, Mean
Absolute Percentage Error (MAPE) digunakan sebagai
metrik evaluasi dalam kompetisi.

In [6]:
# Prediksi pada data uji
y_pred_test = model.predict(test_dmatrix)

# Hitung RMSE dan MAE
rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))
mae = mean_absolute_error(y_test, y_pred_test)
mape = np.mean(np.abs((y_test - y_pred_test) / y_test)) * 100

print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"MAPE: {mape:.2f}%")

MAE: 2915.73
RMSE: 5361.81
MAPE: 9.26%


**Prediksi Data Baru**<br>
Pada tahap prediksi, dilakukan prediksi menggunakan
data baru dari file sample_submission.csv yang telah
disediakan. Data ini terdiri dari dua kolom, yaitu id dan price,
di mana kolom id mencakup informasi mengenai jenis
pangan, provinsi, dan waktu. Dataset ini memiliki 40.664
baris, yang akan digunakan sebagai sampel untuk menguji
performa model dalam melakukan prediksi harga. Pertama,
dilakukan penyesuaian terhadap semua fitur yang digunakan
dalam model pelatihan. Selanjutnya, dataset baru hanya
mempertahankan kolom yang sesuai sebelum dikonversi ke
dalam format DMatrix untuk digunakan dalam prediksi
dengan model. Hasil prediksi kemudian dimasukkan ke
dalam kolom price dan disimpan dalam file CSV untuk
diunggah ke Kaggle.

In [7]:
# Prediksi test
df_new = pd.read_csv('sample_submission.csv')

# Pastikan semua fitur yang digunakan dalam training ada di dataset baru
missing_cols = set(X_train.columns) - set(df_new.columns)
if missing_cols:
    print(f"Peringatan: Kolom berikut tidak ditemukan dalam data baru: {missing_cols}")
    for col in missing_cols:
        df_new[col] = 0  # Isi kolom yang hilang dengan nol agar sesuai

# Pastikan dataset baru hanya menggunakan kolom yang ada di X_train
X_new = df_new[X_train.columns]

# Konversi ke DMatrix dan prediksi
new_dmatrix = xgb.DMatrix(data=X_new)
y_pred_new = model.predict(new_dmatrix)

# Masukkan hasil prediksi ke dalam dataset baru
df_new['price'] = y_pred_new
# Simpan hasil prediksi ke CSV
df_new[['id', 'price']].to_csv("submission.csv", index=False)

df_new

Peringatan: Kolom berikut tidak ditemukan dalam data baru: {'provinsi_Jambi', 'pangan_Gula Konsumsi', 'provinsi_Sulawesi Selatan', 'pangan_Daging Sapi Murni', 'provinsi_Kalimantan Selatan', 'year', 'provinsi_Maluku Utara', 'provinsi_Aceh', 'day', 'pangan_Minyak Goreng Curah', 'provinsi_Papua', 'provinsi_Sumatera Selatan', 'provinsi_Kalimantan Utara', 'provinsi_Sulawesi Tengah', 'provinsi_Papua Barat', 'pangan_Telur Ayam Ras', 'provinsi_Sulawesi Utara', 'provinsi_Kepulauan Bangka Belitung', 'provinsi_Nusa Tenggara Timur', 'pangan_Cabai Merah Keriting', 'provinsi_Sulawesi Barat', 'pangan_Daging Ayam Ras', 'pangan_Tepung Terigu (Curah)', 'pangan_Minyak Goreng Kemasan Sederhana', 'provinsi_DI Yogyakarta', 'provinsi_Maluku', 'provinsi_Banten', 'pangan_Cabai Rawit Merah', 'pangan_Beras Premium', 'provinsi_Gorontalo', 'provinsi_Kepulauan Riau', 'provinsi_Sulawesi Tenggara', 'provinsi_Sumatera Utara', 'provinsi_Kalimantan Barat', 'pangan_Beras Medium', 'provinsi_Sumatera Barat', 'pangan_Bawang

id         price  \
0                         Bawang Merah/Aceh/2024-10-01  17973.380859   
1                         Bawang Merah/Aceh/2024-10-02  17973.380859   
2                         Bawang Merah/Aceh/2024-10-03  17973.380859   
3                         Bawang Merah/Aceh/2024-10-04  17973.380859   
4                         Bawang Merah/Aceh/2024-10-05  17973.380859   
...                                                ...           ...   
40659  Tepung Terigu (Curah)/Sumatera Utara/2024-12-27  17973.380859   
40660  Tepung Terigu (Curah)/Sumatera Utara/2024-12-28  17973.380859   
40661  Tepung Terigu (Curah)/Sumatera Utara/2024-12-29  17973.380859   
40662  Tepung Terigu (Curah)/Sumatera Utara/2024-12-30  17973.380859   
40663  Tepung Terigu (Curah)/Sumatera Utara/2024-12-31  17973.380859   

       provinsi_Jambi  pangan_Gula Konsumsi  provinsi_Sulawesi Selatan  \
0                   0                     0                          0   
1                   0                     0                          0   
2                   0                     0                          0   
3                   0                     0                          0   
4                   0                     0                          0   
...               ...                   ...                        ...   
40659               0                     0                          0   
40660               0                     0                          0   
40661               0                     0                          0   
40662               0                     0                          0   
40663               0                     0                          0   

       pangan_Daging Sapi Murni  provinsi_Kalimantan Selatan  year  \
0                             0                            0     0   
1                             0                            0     0   
2                             0                            0     0   
3                             0                            0     0   
4                             0                            0     0   
...                         ...                          ...   ...   
40659                         0                            0     0   
40660                         0                            0     0   
40661                         0                            0     0   
40662                         0                            0     0   
40663                         0                            0     0   

       provinsi_Maluku Utara  provinsi_Aceh  ...  provinsi_Jawa Tengah  month  \
0                          0              0  ...                     0      0   
1                          0              0  ...                     0      0   
2                          0              0  ...                     0      0   
3                          0              0  ...                     0      0   
4                          0              0  ...                     0      0   
...                      ...            ...  ...                   ...    ...   
40659                      0              0  ...                     0      0   
40660                      0              0  ...                     0      0   
40661                      0              0  ...                     0      0   
40662                      0              0  ...                     0      0   
40663                      0              0  ...                     0      0   

       provinsi_Jawa Timur  provinsi_Nusa Tenggara Barat  \
0                        0                             0   
1                        0                             0   
2                        0                             0   
3                        0                             0   
4                        0                             0   
...                    ...                           ...   
40659                    0                             0   
4066

**Perbandingan Harga Aktual dan Prediksi**<br>
Sampel data diambil berdasarkan
harga komoditas Bawang Merah di Provinsi Aceh pada
berbagai tanggal, dengan harga aktual bervariasi antara
29.200 hingga 35.650, sementara prediksi model berkisar di
angka 30.265 hingga 32.886. Meskipun terdapat perbedaan
antara harga aktual dan prediksi, nilai yang dihasilkan masih
berada dalam rentang yang wajar.

In [8]:
# Buat DataFrame perbandingan data aktual vs prediksi
comparison_df = pd.DataFrame({
    'id': df['id'].iloc[X_test.index],  
    'Actual Price': y_test.values,
    'Predicted Price': y_pred_test
})

comparison_df_sorted = comparison_df.sort_values(by=['id']).reset_index(drop=True)

print(comparison_df_sorted.head(10))


                             id  Actual Price  Predicted Price
0  Bawang Merah/Aceh/2022-01-10       29800.0     30265.417969
1  Bawang Merah/Aceh/2022-01-12       30590.0     30265.417969
2  Bawang Merah/Aceh/2022-01-14       31110.0     30265.417969
3  Bawang Merah/Aceh/2022-01-24       29790.0     30265.417969
4  Bawang Merah/Aceh/2022-01-28       29520.0     30265.417969
5  Bawang Merah/Aceh/2022-02-02       29200.0     31390.523438
6  Bawang Merah/Aceh/2022-02-20       34330.0     32886.308594
7  Bawang Merah/Aceh/2022-02-21       35010.0     32886.308594
8  Bawang Merah/Aceh/2022-02-24       34760.0     32886.308594
9  Bawang Merah/Aceh/2022-02-26       35650.0     32886.308594
